## Simple EDA to understand the dataset
- whats in our dataset
- what do they look like

In [3]:
import pandas as pd
import numpy as np

In [4]:
%%time
train = pd.read_csv('./data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
# note: this process took 2min 10scs on my computer

CPU times: user 2min 10s, sys: 17.2 s, total: 2min 27s
Wall time: 2min 24s


In [5]:
train.head()

,acoustic_data,time_to_failure
0,12,1.4691
1,6,1.4691
2,8,1.4691
3,5,1.4691
4,8,1.4691


In [8]:
train.shape

(629145480, 2)

In [10]:
%%time
sample_sub = pd.read_csv('./data/sample_submission.csv')

CPU times: user 3.82 ms, sys: 2.39 ms, total: 6.22 ms
Wall time: 6.73 ms


In [11]:
sample_sub.head()

,seg_id,time_to_failure
0,seg_00030f,0
1,seg_0012b5,0
2,seg_00184e,0
3,seg_003339,0
4,seg_0042cc,0


In [12]:
sample_test = pd.read_csv('./data/test/seg_0a0fbb.csv')

In [13]:
sample_test.head()

,acoustic_data
0,3
1,-3
2,-1
3,6
4,8


In [14]:
sample_test.shape

(150000, 1)

## LSTM Using Pytorch

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [15]:
# we only use mean and std here just for testing at the current stage
def feature_extraction(time_step):
    return np.c_[time_step.mean(axis=1), time_step.std(axis=1)]

In [16]:
def create_X(x, window_size=1000, seq_len=150):
    X = x.reshape(seq_len,-1)
    return np.c_[feature_extraction(X),
                 feature_extraction(X[:, -window_size // 10:]),
                 feature_extraction(X[:, -window_size // 100:]), ]

In [17]:
features = create_X(train.acoustic_data.values[0:150000]).shape[1]

In [18]:
features

6